In [12]:
import zipfile, shutil
import sqlite3 as sql
import json, os, sys, datetime, re
import csv
import pprint

def get_temp_dir(path, dir_name='temp'):
    temp_dir = os.path.join(path, dir_name)
    temp_dir = os.path.abspath(temp_dir)
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.mkdir(temp_dir)
    return temp_dir


def unpack_apkg(path, unpack_dir=None):
    if not unpack_dir:
        unpack_dir = get_temp_dir(path+'/..', 'unpack')
    db_path = os.path.join(unpack_dir, 'collection.anki2')
    os.mkdir(os.path.join(unpack_dir, 'media'))
    with zipfile.ZipFile(os.path.abspath(path), 'r') as z:
        media = z.read('media').decode()
        media = json.loads(media)
        for i in media:
            fm = z.read(i)
            with open(os.path.join(unpack_dir+'/media', media[i]), 'wb') as f:
                f.write(fm)
        db = z.read('collection.anki2')
        with open(os.path.join(unpack_dir, 'collection.anki2'), 'wb') as f:
            f.write(db)
            
    db_path = os.path.abspath(db_path)
    with sql.connect(db_path) as dbconn:
        cursor = dbconn.cursor()
        models = cursor.execute('select models from col').fetchone()
        models = json.loads(models[0])
        models = tuple(models.values())[0]
        
    
        flds_name = sorted(models['flds'],key=lambda x: x['ord'])
        flds_name = list(map(lambda x: x['name'], flds_name))
        flds_name.append('tags')
            
            
        notes = cursor.execute('select flds, tags from notes').fetchall()
        notes = list(map(lambda x: x[0].split('\x1f') + [x[1].strip()], notes))
    
    
    with open(os.path.join(unpack_dir, 'cards.css'), 'w') as f:
        f.write(models['css'])
    
    for tmpl in models['tmpls']:
        with open(os.path.join(unpack_dir, '{}.txt'.format(tmpl['name'])), 'w') as f:
            f.write(tmpl['qfmt'])
            f.write('\n<====================>\n')
            f.write('<====================>\n')
            f.write(tmpl['afmt'])
            
    with open(os.path.join(unpack_dir, 'notes.csv'), 'w') as f:
        w = csv.writer(f, dialect='excel-tab')
        w.writerow(flds_name)
        w.writerows(notes)
        #=w=
        


unpack_apkg('collection.apkg')
def package_media(path, temp_dir=None, ex={'collection.anki2'}):
    if not temp_dir:
        temp_dir = get_temp_dir(path+'/..')
    media = dict()
    media_files = os.listdir(path)
    media_files = filter(lambda x: x not in ex, media_files)
    for i, m in enumerate(media_files):
        i = str(i)
        media[i] = m
        shutil.copyfile(os.path.join(path, m), os.path.join(temp_dir, i))
    with open(os.path.join(temp_dir, 'media'), 'w') as f:
        json.dump(media, f)


NEW_DB = '''
CREATE TABLE col (
    id              integer primary key,
    crt             integer not null,
    mod             integer not null,
    scm             integer not null,
    ver             integer not null,
    dty             integer not null,
    usn             integer not null,
    ls              integer not null,
    conf            text not null,
    models          text not null,
    decks           text not null,
    dconf           text not null,
    tags            text not null
);
CREATE TABLE notes (
    id              integer primary key,   /* 0 */
    guid            text not null,         /* 1 */
    mid             integer not null,      /* 2 */
    mod             integer not null,      /* 3 */
    usn             integer not null,      /* 4 */
    tags            text not null,         /* 5 */
    flds            text not null,         /* 6 */
    sfld            integer not null,      /* 7 */
    csum            integer not null,      /* 8 */
    flags           integer not null,      /* 9 */
    data            text not null          /* 10 */
);
CREATE TABLE cards (
    id              integer primary key,   /* 0 */
    nid             integer not null,      /* 1 */
    did             integer not null,      /* 2 */
    ord             integer not null,      /* 3 */
    mod             integer not null,      /* 4 */
    usn             integer not null,      /* 5 */
    type            integer not null,      /* 6 */
    queue           integer not null,      /* 7 */
    due             integer not null,      /* 8 */
    ivl             integer not null,      /* 9 */
    factor          integer not null,      /* 10 */
    reps            integer not null,      /* 11 */
    lapses          integer not null,      /* 12 */
    left            integer not null,      /* 13 */
    odue            integer not null,      /* 14 */
    odid            integer not null,      /* 15 */
    flags           integer not null,      /* 16 */
    data            text not null          /* 17 */
);
CREATE TABLE revlog (
    id              integer primary key,
    cid             integer not null,
    usn             integer not null,
    ease            integer not null,
    ivl             integer not null,
    lastIvl         integer not null,
    factor          integer not null,
    time            integer not null,
    type            integer not null
);
CREATE TABLE graves (
    usn             integer not null,
    oid             integer not null,
    type            integer not null
);
CREATE INDEX ix_notes_usn on notes (usn);
CREATE INDEX ix_cards_usn on cards (usn);
CREATE INDEX ix_revlog_usn on revlog (usn);
CREATE INDEX ix_cards_nid on cards (nid);
CREATE INDEX ix_cards_sched on cards (did, queue, due);
CREATE INDEX ix_revlog_cid on revlog (cid);
CREATE INDEX ix_notes_csum on notes (csum);
'''


def new_db(path):
    db_path = os.path.abspath(os.path.join(path, 'collection.anki2'))
    if os.path.exists(db_path):
        os.remove(db_path)
    with sql.connect(db_path) as dbconn:
        cursor = dbconn.cursor()
        cursor.executescript(NEW_DB)

        id = 1
        mod = int(datetime.datetime.now().timestamp()*1000)
        crt = mod // 1000
        # crt = 1470081600
        scm = mod
        ver = 1
        dty = 0
        usn = 0
        ls  = 0
        
        conf = '{}'
        dconf = '{}'
        tags = '{}'
        #############################
        
        

        
        
        
        css = ""
        models = None
        decks = None
        

# new_db('.')
def gen_from_apkg(apkg_path, temp_dir=None):
    if not temp_dir:
        temp_dir = get_temp_dir(path+'/..')


# package_media('unpack/media')


In [45]:
import zipfile, shutil
import sqlite3 as sql
import json, os, sys, datetime, re
import csv
import pprint

class AnkiPackageMaker(object):
    
    def __init__(self, config=dict()):
        '''temp_dir work_dir 可选'''
        self.config = config
        default_config = {
            'temp_dir'  :'',
            'work_dir'  :'',
        }

        for (k, v) in default_config.items():
            self.config[k] = config[k] if config.get(k) else v
                
        temp_dir = self.config['temp_dir']
        if temp_dir == '':
            self.config['temp_dir'] = tempfile.TemporaryDirectory()
        else:
            self.config['temp_dir'] = tempfile.TemporaryDirectory(dir=temp_dir)
            
    def unpack(self, src_path, unpack_dir):
        with zipfile.ZipFile(src_path, 'r') as z:
            
            #unpack media
            media = z.read('media').decode()
            media = json.loads(media)
            
            media_dir = os.path.join(unpack_dir, 'media')
            os.makedirs(media_dir)
            for i in media:
                fm = z.read(i)
                media_path = os.path.join(media_dir, media[i])
                with open(media_path, 'wb') as f:
                    f.write(fm)
                    
            #unpack others
            db = z.read('collection.anki2')
            
            #debug
            with open(os.path.join(unpack_dir, 'collection.anki2'), 'wb') as f:
                f.write(db)
            #end debug
            
            with tempfile.NamedTemporaryFile() as f:
                f.file.write(db)
                f.file.close()
                with sql.connect(f.name) as dbconn:
                    # 获取设置
                    cursor = dbconn.cursor()
                    models = cursor.execute('select models from col').fetchone()
                    models = json.loads(models[0])
                    
                    #unpack each model
                    for (mid, model) in models.items():
                        model_dir = os.path.join(unpack_dir, model['name'])
                        os.makedirs(model_dir)
                        # get csv headline
                        flds_name = sorted(model['flds'],key=lambda x: x['ord'])
                        flds_name = list(map(lambda x: x['name'], flds_name))
                        flds_name.append('tags')
                        # csv content
                        notes = cursor.execute('select flds, tags \
                                               from notes \
                                               where mid = ?',
                                               (mid,)).fetchall()
                        notes = list(map(lambda x: x[0].split('\x1f') + [x[1].strip()], notes))
                        # write to file
                        with open(os.path.join(model_dir, 'notes.csv'), 'w') as f:
                            w = csv.writer(f, dialect='excel-tab')
                            w.writerow(flds_name)
                            w.writerows(notes)
                        # tmpls
                        for tmpl in model['tmpls']:
                            with open(os.path.join(model_dir, '{}.txt'.format(tmpl['name'])), 'w') as f:
                                f.write(tmpl['qfmt'])
                                f.write('\n<====================>\n')
                                f.write('<====================>\n')
                                f.write(tmpl['afmt'])
                        # css
                        with open(os.path.join(model_dir, 'cards.css'), 'w') as f:
                            f.write(model['css'])

                            
    def package(self, taget_path, deck_name, package_list, media_dir):
        '''
        package_list 
            'name'      :'',
            'csv_path'  :'',
            'css_path'  :'',
            'tmpls_dict' :[],
        '''
        
        
        def timestamp():
            return int(datetime.datetime.now().timestamp())
        
        def msstamp():
            return int(datetime.datetime.now().timestamp()*1000)
        
        def make_conf(curModel=None, curDeck=1, activeDecks=None):
            activeDecks = activeDecks if activeDecks else [curDeck]
            conf = {
                'activeDecks': activeDecks,
                'addToCur': True,
                'collapseTime': 1200,
                'curDeck': curDeck,
                'curModel': curModel if curModel else str(msstamp()),
                'dueCounts': True,
                'estTimes': True,
                'newBury': True,
                'newSpread': 0,
                'nextPos': 1,
                'sortBackwards': False,
                'sortType': 'noteFld',
                'timeLim': 0}
            return conf
        
        def make_dconf(deck_id=1):
            return {'{}'.format(deck_id):
                    {'autoplay': True,
                     'id': 1,
                     'lapse': {
                        'delays': [10],
                        'leechAction': 0,
                        'leechFails': 8,
                        'minInt': 1,
                        'mult': 0},
                     'maxTaken': 60,
                     'mod': 0,
                     'name': 'Default',
                     'new': {
                        'bury': True,
                        'delays': [1, 10],
                        'initialFactor': 2500,
                        'ints': [1, 4, 7],
                        'order': 1,
                        'perDay': 20,
                        'separate': True},
                     'replayq': True,
                     'rev': {
                        'bury': True,
                        'ease4': 1.3,
                        'fuzz': 0.05,
                        'ivlFct': 1,
                        'maxIvl': 36500,
                        'minSpace': 1,
                        'perDay': 100},
                     'timer': 0,
                     'usn': 0}}
        
        
        # make col
        def make_col(models, decks=None, conf=None, tags=None, dconf=None, crt=None):
            crt = crt if crt else timestamp()
            col = {
                'id' = 1,
                'crt' = crt,
                'mod' = crt * 1000,
                'scm' = crt * 1000,
                'ver' = 1,
                'dty' = 0,
                'usn' = 0,
                'ls'  = 0,
                'conf'= conf if conf else make_conf(),
                'models' = models,
                'decks' = decks,
                'dconf' = dconf if dconf else make_dconf(),
                'tags'  = tags if tags else {},
            }
        
        for (name, csv_path, css_path, tmpls_dict) in package_list:
            pass
#         def gen_models(model_name, flds, tmpls, css=CSS):
#             model_id = now_time()
#             desk_id  = now_time()

#             req = list([[i, "any", [0]] for i in range(len(tmpls))])
    
#             flds = gen_flds(flds)
#             models = {
#                     model_id : {
#                         "vers": [],
#                         "name": model_name,
#                         "tags": [],
#                         "did": desk_id,
#                         "usn": 0,
#                         "req": req,
#                         "flds": flds,
#                         "sortf": 0,
#                         "tmpls": gen_tmpls(tmpls, desk_id),
#                         "mod": model_id[:-3],
#                         "latexPost": "\\end{document}",
#                         "type": 0,
#                         "id": model_id,
#                         "css": css,
#                         "latexPre": "\\documentclass[12pt]{article}\n\\special{papersize=3in,5in}\n\\usepackage[utf8]{inputenc}\n\\usepackage{amssymb,amsmath}\n\\pagestyle{empty}\n\\setlength{\\parindent}{0in}\n\\begin{document}\n"
#                     }
#                 }
    
    def __del__(self):
        self.config['temp_dir'].cleanup()

maker = AnkiPackageMaker()
# %rm -r aaaaaaaaaaaaa
# maker.unpack('a.apkg', 'aaaaaaaaaaaaa')
%rm -r xxx
maker.unpack('xxx.apkg', 'xxx')

In [40]:
rm -r aaaaaaaaaaaaa

In [11]:
def gen_tmpls(tmpls_dict, did = None):
    '''{"卡片 1": "text1", "卡片 2": "text2"}'''
    tmpls = []
    for (i, (name, text)) in enumerate(tmpls_dict.items()):
        tmpl = {
                'afmt' : "",
                'bafmt' : "",
                'bqfmt' : "",
                'did' : did,
                'name' : "{}".format(name),
                'ord' : i,
                'qfmt' : ""
              }
        m = re.fullmatch(r'(.*)(\n[<]={10,}[>])\2\n(.*)',
                         text, flags=re.DOTALL)
        if m:
            r = m.groups()
            if len(r) == 3:
                tmpl["qfmt"] = r[0]
                tmpl["afmt"] = r[2]
        tmpls.append(tmpl)
        
    if tmpls:
        tmpls[0]['did'] = None
    return tmpls

[{'afmt': '{{图片}}<br>\n{{键位}}',
  'bafmt': '',
  'bqfmt': '',
  'did': None,
  'name': '卡片1',
  'ord': 0,
  'qfmt': '{{字母}} <br>\n{{键位}}\n<====================>\n<====================>\n{{字母}} {{识别码}} {{一级简码}}\n<hr id=answer>\n{{口诀}} <br>\n<===>\n<===>'},
 {'afmt': '{{FrontSide}}\n\n<hr id=answer>\n\n{{字母}} {{识别码}} {{一级简码}} <br>\n{{图片}}<br>\n{{键位}}',
  'bafmt': '',
  'bqfmt': '',
  'did': None,
  'name': '卡片 2',
  'ord': 1,
  'qfmt': '{{口诀}}'}]

In [10]:
def unicode_escape(string):
    return string.encode('unicode_escape').decode()

def gen_flds(flds):
    return list([{
                "ord": i,
                "name": name,
                "media": [],
                "sticky": False,
                "rtl": False,
                "font": "Arial",
                "size": 20
            }
            for i, name in enumerate(flds)])

def gen_tmpl(qfmt, afmt, name, desk_id, order):
    return {
        "name": name,
        "qfmt": qfmt,
        "did": desk_id,
        "bafmt": "",
        "afmt": afmt,
        "ord": order,
        "bqfmt": ""
    }

def gen_tmpls(tmpls, desk_id):
    tmpls = list([gen_tmpl(tmpl['qfmt'],
                           tmpl['afmt'],
                           tmpl['name'] if tmpl.get('name') else now_time(),
                           desk_id,
                           order)
                  for order, tmpl in enumerate(tmpls)])
    tmpls[0]['did'] = None
    return tmpls

CSS = '.card {\n font-family: arial;\n font-size: 20px;\n text-align: center;\n color: black;\n background-color: white;\n}\n'

def now_time():
    return str(int(datetime.datetime.now().timestamp()*1000))

def gen_models(model_name, flds, tmpls, css=CSS):
    model_id = now_time()
    desk_id  = now_time()
    
    req = list([[i, "any", [0]] for i in range(len(tmpls))])
    
    flds = gen_flds(flds)
    models = {
            model_id : {
                "vers": [],
                "name": model_name,
                "tags": [],
                "did": desk_id,
                "usn": 0,
                "req": req,
                "flds": flds,
                "sortf": 0,
                "tmpls": gen_tmpls(tmpls, desk_id),
                "mod": model_id[:-3],
                "latexPost": "\\end{document}",
                "type": 0,
                "id": model_id,
                "css": css,
                "latexPre": "\\documentclass[12pt]{article}\n\\special{papersize=3in,5in}\n\\usepackage[utf8]{inputenc}\n\\usepackage{amssymb,amsmath}\n\\pagestyle{empty}\n\\setlength{\\parindent}{0in}\n\\begin{document}\n"
            }
        }
    return models

In [53]:
def insert_note(row):
    pass

def read_csv(filename, encoding = 'gbk'):
    with open(filename, 'r', encoding=encoding) as csvfile:
        reader = csv.DictReader(csvfile,dialect="excel-tab")
        flds = gen_flds(reader.fieldnames)
        for row in reader:
            insert_note(row)

In [11]:
gen_models('测试', ['字母','识别码','一级简码','口诀','图片','键位'], 
           [{'qfmt': '{{字母}} <br>\n{{键位}}', 'afmt': '{{字母}} {{识别码}} {{一级简码}}\n'
          '<hr id=answer>\n'
          '{{口诀}} <br>\n'
          '{{图片}}<br>\n'
          '{{键位}}', 'name': '卡片 1'}])

{'1485613750040': {'css': '.card {\n font-family: arial;\n font-size: 20px;\n text-align: center;\n color: black;\n background-color: white;\n}\n',
  'did': '1485613750040',
  'flds': [{'font': 'Arial',
    'media': [],
    'name': '字母',
    'ord': 0,
    'rtl': False,
    'size': 20,
    'sticky': False},
   {'font': 'Arial',
    'media': [],
    'name': '识别码',
    'ord': 1,
    'rtl': False,
    'size': 20,
    'sticky': False},
   {'font': 'Arial',
    'media': [],
    'name': '一级简码',
    'ord': 2,
    'rtl': False,
    'size': 20,
    'sticky': False},
   {'font': 'Arial',
    'media': [],
    'name': '口诀',
    'ord': 3,
    'rtl': False,
    'size': 20,
    'sticky': False},
   {'font': 'Arial',
    'media': [],
    'name': '图片',
    'ord': 4,
    'rtl': False,
    'size': 20,
    'sticky': False},
   {'font': 'Arial',
    'media': [],
    'name': '键位',
    'ord': 5,
    'rtl': False,
    'size': 20,
    'sticky': False}],
  'id': '1485613750040',
  'latexPost': '\\end{document}',

In [12]:
c = gen_flds(["\u5b57\u6bcd", "\u8bc6\u522b\u7801", "\u4e00\u7ea7\u7b80\u7801", "\u53e3\u8bc0", "\u56fe\u7247", "\u952e\u4f4d"])

In [7]:
with open(r'unpack/models.json', 'r') as f:
    x=json.loads(f.read())
    d= next(iter(x.values()))['tmpls']
    pprint.pprint(d)

[{'afmt': '{{字母}} {{识别码}} {{一级简码}}\n'
          '<hr id=answer>\n'
          '{{口诀}} <br>\n'
          '{{图片}}<br>\n'
          '{{键位}}',
  'bafmt': '',
  'bqfmt': '',
  'did': None,
  'name': '卡片 2',
  'ord': 0,
  'qfmt': '{{字母}} <br>\n{{键位}}'},
 {'afmt': '{{FrontSide}}\n'
          '\n'
          '<hr id=answer>\n'
          '\n'
          '{{字母}} {{识别码}} {{一级简码}} <br>\n'
          '{{图片}}<br>\n'
          '{{键位}}',
  'bafmt': '',
  'bqfmt': '',
  'did': 1470123357010,
  'name': '卡片 3',
  'ord': 1,
  'qfmt': '{{口诀}}'}]


In [8]:
with open(r'unpack/models.json', 'r') as f:
    x=json.loads(f.read())
    d= next(iter(x.values()))['flds']
    pprint.pprint(d)

[{'font': 'Arial',
  'media': [],
  'name': '字母',
  'ord': 0,
  'rtl': False,
  'size': 20,
  'sticky': False},
 {'font': 'Arial',
  'media': [],
  'name': '识别码',
  'ord': 1,
  'rtl': False,
  'size': 20,
  'sticky': False},
 {'font': 'Arial',
  'media': [],
  'name': '一级简码',
  'ord': 2,
  'rtl': False,
  'size': 20,
  'sticky': False},
 {'font': 'Arial',
  'media': [],
  'name': '口诀',
  'ord': 3,
  'rtl': False,
  'size': 20,
  'sticky': False},
 {'font': 'Arial',
  'media': [],
  'name': '图片',
  'ord': 4,
  'rtl': False,
  'size': 20,
  'sticky': False},
 {'font': 'Arial',
  'media': [],
  'name': '键位',
  'ord': 5,
  'rtl': False,
  'size': 20,
  'sticky': False}]


[{'font': 'Arial',
  'media': [],
  'name': '字母',
  'ord': 0,
  'rtl': False,
  'size': 20,
  'sticky': False},
 {'font': 'Arial',
  'media': [],
  'name': '识别码',
  'ord': 1,
  'rtl': False,
  'size': 20,
  'sticky': False},
 {'font': 'Arial',
  'media': [],
  'name': '一级简码',
  'ord': 2,
  'rtl': False,
  'size': 20,
  'sticky': False},
 {'font': 'Arial',
  'media': [],
  'name': '口诀',
  'ord': 3,
  'rtl': False,
  'size': 20,
  'sticky': False},
 {'font': 'Arial',
  'media': [],
  'name': '图片',
  'ord': 4,
  'rtl': False,
  'size': 20,
  'sticky': False},
 {'font': 'Arial',
  'media': [],
  'name': '键位',
  'ord': 5,
  'rtl': False,
  'size': 20,
  'sticky': False}]

In [ ]:
cursor = dbconn.cursor

In [2]:
bytes(r"\u53e3\u8bc0", 'utf8').decode('unicode_escape')

'口诀'

In [9]:
cd d:\playground\csv2anki

d:\playground\csv2anki


In [33]:
import tempfile

with tempfile.NamedTemporaryFile(mode='w') as fp:
    fp.file.write('9')
    fp.file.close()
    with open(fp.name) as f:
        print(f.read())

9
